In [1]:
import pandas as pd
import keplergl
import json
import csv
import geopandas
import matplotlib.pyplot as plt
import numpy as np

## I redid my implementation to be more akin to Kylie's as that method seems to behave better on my machine

In [2]:
## Ian's Code
def read_partition(partition_fname):
    """
    Reads a partition (district map) from a csv file with 
    columns 'GEOID' (unique identifiers for the census tracts) and 'district'. 
    
    Returns a pandas DataFrame object representing the map, where
    'GEOID' is the key.
    """
    with open(partition_fname, 'r') as map_file:
        map_reader = csv.reader(map_file)
        map_raw = list(map_reader)

    map_headers = map_raw.pop(0)
    map_df = pd.DataFrame(map_raw, columns=map_headers).astype({'district': int})
    return map_df.set_index('GEOID')


def get_sample_wi_map(tracts_fname, partition_fname):
    """
    Loads a sample Wisconsin district map. 
    Parameters:
        tracts_fname: the name of the Wisconsin census tracts file (zipped shapefile)
        partition_fname: the name of the initial partition file (.csv)
    Returns:
        the given Wisconsin district map as a GerryChain Partition object
    """
    # Step 1. Build a GeoDataFrame (a geographic version of a pandas DataFrame)
    # from the tracts zip file (adding 'zip://' prefix if missing). 
    tracts_fname = tracts_fname if 'zip://' in tracts_fname else 'zip://' + tracts_fname
    gdf = geopandas.read_file(tracts_fname)
    gdf.set_index('GEOID', inplace=True)

    # Step 2. Load the initial partition and join it to the GeoDataFrame. 
    map_df = read_partition(partition_fname)
    map_gdf = gdf.join(map_df)

    map_gdf['district'].fillna(value=-1, inplace=True) # Slight cleanup
    
    return map_gdf

In [3]:
if __name__ == '__main__':
    tracts_fname = 'tl_2013_55_tract.zip'
    partition_fname = 'wi_gerrymander_dem.csv'
    sample_map = get_sample_wi_map(tracts_fname, partition_fname)


In [12]:
#flip_json = '[{"55081950200": "5"},{"55035000100": "1"},{"55097961000": "1"},{"55133204400": "5"},{"55055100900": "5"},{"55133204400": "6"},{"55133204400": "2"},{"55021970600": "3"},{"55055101500": "5"},{"55039041500": "7"},{"55097961200": "7"}]'

In [4]:
#parse data into json
with open('redist-vis/data/wi_path_10flips.json') as flips_json:
    path_data = json.load(flips_json)


In [6]:
flips = path_data['flips']

In [7]:
initial_map = path_data['initial_map']

In [23]:
    #make kepler map
    ##test_map = keplergl.KeplerGl()
    ##test_map.add_data(data=sample_map, name="districts")
    #sample_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [8]:
#update districts to be those from initial map
initial_gdf = sample_map
for key in initial_map:
    for geoid in initial_map[key]:
        initial_gdf.loc[geoid,'district'] = key


In [16]:
initial_gdf.geometry.name

'geometry'

In [40]:
#THIS SHOULD BE RUN SECOND TO LAST
test_map = keplergl.KeplerGl()
test_map.add_data(data=initial_gdf, name="initial_map")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [41]:
#THIS SHOULD BE RUN LAST
test_map.save_to_html(file_name='test_map_2.html')

Map saved to test_map_2.html!


In [38]:
#add a layer for each flip
for i in range(len(flips)):
    initial_gdf['layer_' + str(i + 1)] = initial_gdf['district']
    for key in flips[i]:
        initial_gdf.loc[key, 'layer_' + str(i + 1)] = flips[i][key]
    

In [39]:
initial_gdf

,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,...,layer_2,layer_3,layer_4,layer_5,layer_6,layer_7,layer_8,layer_9,layer_10,layer_11
GEOID,,,,,,,,,,,,,,,,,,,,,
55009940002,55,009,940002,9400.02,Census Tract 9400.02,G5020,S,11517097,0,+44.5145501,...,7,7,7,7,7,7,7,7,7,7
55009001600,55,009,001600,16,Census Tract 16,G5020,S,3660554,33519,+44.4885958,...,7,7,7,7,7,7,7,7,7,7
55009001801,55,009,001801,18.01,Census Tract 18.01,G5020,S,11447611,45849,+44.5146596,...,7,7,7,7,7,7,7,7,7,7
55009001802,55,009,001802,18.02,Census Tract 18.02,G5020,S,19321287,20945836,+44.5246043,...,7,7,7,7,7,7,7,7,7,7
55009002001,55,009,002001,20.01,Census Tract 20.01,G5020,S,6978578,0,+44.4979025,...,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55043960800,55,043,960800,9608,Census Tract 9608,G5020,S,253212075,15985750,+42.7117442,...,3,3,3,3,3,3,3,3,3,3
55043960100,55,043,960100,9601,Census Tract 9601,G5020,S,364751521,3687082,+43.0851261,...,3,3,3,3,3,3,3,3,3,3
55043960500,55,043,960500,9605,Census Tract 9605,G5020,S,569009923,31227383,+42.8066244,...,3,3,3,3,3,3,3,3,3,3
